## **Transfer Learning et Fine-Tuning PASCAL VOC2007**

In [0]:
###Exercice 2 : Perceptron avec Keras
!pip install  --upgrade grpcio
!pip uninstall tensorflow==1.15.0
!pip install tensorflow-gpu==2.0.0

## **Exercice 1 : Modèle ResNet-50 avec Keras**

In [0]:
from tensorflow.keras.models import Sequential
# from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Activation, Input
from tensorflow.keras.optimizers import SGD
from keras.utils import np_utils 
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50
model = ResNet50(include_top=True, weights='imagenet')


# *Exercice* 2 : Extraction de « Deep Features »

In [0]:
model.summary()

In [0]:
from tensorflow.keras.models import Model

# model.layers.pop()
model = Model(inputs=model.inputs , outputs=model.layers[-2].output)

In [0]:
model.summary()
model.compile(loss='binary_crossentropy', optimizer=SGD(lr =0.01, momentum=0.9), metrics=['binary_accuracy'])

In [0]:
import zipfile
from google.colab import drive


!wget  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar


!mkdir -p ./content/data
!tar xvf VOCtrainval_06-Nov-2007.tar -C ./content/data/   # -C ./datasets/trainset
!tar xvf VOCtest_06-Nov-2007.tar -C ./content/data/  #-C ./datasets/testset




In [0]:
import os
import random
import numpy as np
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input

random.seed(2506)

IMG_HEIGHT = 224
IMG_WIDTH = 224

LABELS = ['aeroplane', 'bicycle', 'bird', 'boat',
           'bottle', 'bus', 'car', 'cat', 'chair',
           'cow', 'diningtable', 'dog', 'horse',
           'motorbike', 'person', 'pottedplant',
           'sheep', 'sofa', 'train', 'tvmonitor']


class PascalVOCDataGenerator(object):
    """
    PascalVOCDataGenerator defines a generator on the PascalVOC 2007 dataset 
    
    Here are the links to download the data :
    val and train  :  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
    test           :  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
    """

    def __init__(self, subset, data_path):
        
        assert subset in ['train', 'val', 'trainval', 'test']
        self.subset = subset

        self.data_path = data_path
        self.images_path = os.path.join(self.data_path, 'JPEGImages')
        self.labels_path = os.path.join(self.data_path, 'ImageSets', 'Main')

        # The id_to_label dict has the following structure
        # key : image's id (e.g. 00084)
        # value : image's label (e.g. [0, 0, 1, 0, ..., 1, 0])
        self.id_to_label = {}

        self.labels = LABELS
        self.nb_classes = len(self.labels) # 20 classes for PascalVOC

        # Get all the images' ids for the given subset
        self.images_ids_in_subset = self._get_images_ids_from_subset(self.subset)
        
        # Create the id_to_label dict with all the images' ids 
        # but the values are arrays with nb_classes (20) zeros 
        self._initialize_id_to_label_dict()
        
        # Fill the values in the id_to_label dict by putting 1 when
        # the label is in the image given by the key
        self._fill_id_to_label_dict_with_classes()

    def _initialize_id_to_label_dict(self):
        for image_id in self.images_ids_in_subset:
            self.id_to_label[image_id] = np.zeros(self.nb_classes)

    def _fill_id_to_label_dict_with_classes(self):
        """_fill_id_to_label_dict_with_classes
        For each class, the <class>_<subset>.txt file contain the presence information
        of this class in the image
        """
        for i in range(self.nb_classes):
            label = self.labels[i]
            # Open the <class>_<subset>.txt file
            with open(os.path.join(self.labels_path, "%s_%s.txt" % (label, self.subset)), 'r') as f:
                lines = f.read().splitlines()
                for line in lines:
                    splited_line = line.split()
                    image_id = splited_line[0]
                    is_present = int(splited_line[1])
                    if is_present == 1:
                        self.id_to_label[image_id][i] = 1

    def _get_images_ids_from_subset(self, subset):
        """_get_images_ids_from_subset
        The images' ids are found in the <subset>.txt file in ImageSets/Main
        """
        with open(os.path.join(self.labels_path, subset + '.txt'), 'r') as f:
            images_ids = f.read().splitlines()
        return images_ids

    def flow(self, batch_size=32):
        """flow
        This is a generator which load the images and preprocess them on the fly
        When calling next python build in function, it returns a batch with a given size
        with a X_batch of size (None, IMG_HEIGHT, IMG_WIDTH, 3)
        and a Y_batch of size (None, nb_classes)
        The first dimension is the batch_size if there is enough images left otherwise 
        it will be less

        :param batch_size: the batch's size
        """
        nb_batches = int(len(self.images_ids_in_subset) / batch_size) + 1
        while True:
            # Before each epoch we shuffle the images' ids
            random.shuffle(self.images_ids_in_subset)
            for i in range(nb_batches):
                # We first get all the images' ids for the next batch
                current_bach = self.images_ids_in_subset[i*batch_size:(i+1)*batch_size]
                X_batch = []
                Y_batch = []
                for image_id in current_bach:
                    # Load the image and resize it. We get a PIL Image object 
                    img = image.load_img(os.path.join(self.images_path, image_id + '.jpg'), grayscale=False, target_size=(IMG_HEIGHT, IMG_WIDTH))
                    # Cast the Image object to a numpy array and put the channel has the last dimension
                    img_arr = image.img_to_array(img, data_format='channels_last')
                    X_batch.append(img_arr)
                    Y_batch.append(self.id_to_label[image_id])
                # resize X_batch in (batch_size, IMG_HEIGHT, IMG_WIDTH, 3) 
                X_batch = np.reshape(X_batch, (-1, IMG_HEIGHT, IMG_WIDTH, 3))
                # resize Y_batch in (None, nb_classes) 
                Y_batch = np.reshape(Y_batch, (-1, self.nb_classes))
                # The preprocess consists of substracting the ImageNet RGB means values
                # https://github.com/keras-team/keras/blob/master/keras/applications/imagenet_utils.py#L72
                X_batch = preprocess_input(X_batch, data_format='channels_last')
                yield(X_batch, Y_batch)


In [0]:
# import PascalVOCDataGenerator

data_dir  = './content/data/VOCdevkit/VOC2007' # A changer avec votre chemin
data_generator_train = PascalVOCDataGenerator('trainval', data_dir)

# trainset_dir = 'VOCdevkit/VOC2007' # A changer avec votre chemin
# data_generator_train = PascalVOCDataGenerator('trainval', trainset_dir)

# testset_dir = '/content/VOCdevkit/VOC2007' # A changer avec votre chemin
# data_generator_test = PascalVOCDataGenerator('test', testset_dir)


In [0]:
# generator = data_generator_train.flow(batch_size=64)


In [0]:
batch_size = 32
generator = data_generator_train.flow(batch_size=batch_size)
# Initilisation des matrices contenant les Deep Features et les labels
X_train = np.zeros((len(data_generator_train.images_ids_in_subset),2048))
Y_train = np.zeros((len(data_generator_train.images_ids_in_subset),20))
# Calcul du nombre e batchs
nb_batches = int(len(data_generator_train.images_ids_in_subset) / batch_size) + 1

for i in range(nb_batches):
    # Pour chaque batch, on extrait les images d'entrée X et les labels y
    X, y = next(generator)
    # On récupère les Deep Feature par appel à predict
    y_pred = model.predict(X)
    print("y_pred shape: ",y_pred.shape)
    print("X_train shape: ",X_train.shape)
    # break
    X_train[i*batch_size:(i+1)*batch_size,:] = y_pred
    Y_train[i*batch_size:(i+1)*batch_size,:] = y

In [0]:
data_dir  = './content/data/VOCdevkit/VOC2007' # A changer avec votre chemin
data_generator_test = PascalVOCDataGenerator('test', data_dir)


batch_size = 32
generator = data_generator_test.flow(batch_size=batch_size)
# Initilisation des matrices contenant les Deep Features et les labels
X_test = np.zeros((len(data_generator_test.images_ids_in_subset),2048))
Y_test = np.zeros((len(data_generator_test.images_ids_in_subset),20))
# Calcul du nombre e batchs
nb_batches = int(len(data_generator_test.images_ids_in_subset) / batch_size) + 1

for i in range(nb_batches):
    # Pour chaque batch, on extrait les images d'entrée X et les labels y
    X, y = next(generator)
    # On récupère les Deep Feature par appel à predict
    y_pred = model.predict(X)
    print("y_pred shape: ",y_pred.shape)
    print("X_test shape: ",X_test.shape)
    # break
    X_test[i*batch_size:(i+1)*batch_size,:] = y_pred
    Y_test[i*batch_size:(i+1)*batch_size,:] = y

In [0]:
outfile = 'DF_ResNet50_VOC2007'
# Initilisation des matrices contenant les Deep Features et les labels
# X_test = np.zeros((len(data_generator_test.images_ids_in_subset),2048))
# Y_test = np.zeros((len(data_generator_test.images_ids_in_subset),20))
np.savez(outfile, X_train=X_train, Y_train=Y_train,X_test=X_test, Y_test=Y_test)

# **Exercice 3 : Transfert sur VOC 2007**

In [0]:
import numpy as np
from sklearn.metrics import average_precision_score
#from data_gen import PascalVOCDataGenerator
###Transfer Learning
outfile = '/content/DF_ResNet50_VOC2007.npz'
npzfile = np.load(outfile)
X_train = npzfile['X_train']
Y_train = npzfile['Y_train']
X_test = npzfile['X_test']
Y_test = npzfile['Y_test']
print ("X_train=",X_train.shape, "Y_train=",Y_train.shape, " X_test=",X_test.shape, "Y_train=",Y_test.shape)

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()
model.add(Dense(20,  input_dim=2048, name='fc1', activation='sigmoid'))
model.summary()


In [0]:
from tensorflow.keras.optimizers import SGD
learning_rate = 0.1
sgd = SGD(learning_rate)
model.compile(loss='binary_crossentropy',optimizer=sgd,metrics=['binary_accuracy'])

In [0]:
batch_size = 32
nb_epoch = 20
model.fit(X_train, Y_train,batch_size=batch_size, epochs=nb_epoch,verbose=1)
scores = model.evaluate(X_test, Y_test, verbose=0)
print("%s TEST: %.2f%%" % (model.metrics_names[0], scores[0]*100))
print("%s TEST: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [0]:
from sklearn.metrics import average_precision_score

y_pred_test = model.predict(X_test)
#y_pred_test = np.argmax(y_pred_test,axis=1)
y_pred_train = model.predict(X_train)
#y_pred_train = model.argmax(y_pred_train,axis=1)
AP_train = np.zeros(20)
AP_test = np.zeros(20)
for c in range(20):
    AP_train[c] = average_precision_score(Y_train[:, c], y_pred_train[:, c])
    AP_test[c] = average_precision_score(Y_test[:, c], y_pred_test[:, c])

print ("MAP TRAIN =", AP_train.mean()*100)
print ("MAP TEST =", AP_test.mean()*100)

# **Exercice 4 : Fine-tuning sur VOC 2007**

In [0]:
import tensorflow as tf
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

In [0]:
# Load ResNet50 architecture & its weights
# model = Model(inputs=model.inputs , outputs=model.layers[-2].output)

model = ResNet50(include_top=True, weights='imagenet')
model.layers.pop()
# Modify top layers
x = model.layers[-1].output
x = Dense(data_generator_train.nb_classes, activation='sigmoid', name='predictions')(x)
model = Model(inputs=model.inputs,outputs=x)

Freezing the layers except the last 4 layers

for layer in conv_base.layers[:-4]:

layer.trainable = False

In [0]:
# model.summary()
for i in range(len(model.layers)):
  model.layers[i].trainable = True

In [0]:
lr = 0.1
model.compile(loss='binary_crossentropy', optimizer=SGD(lr=lr), metrics=['binary_accuracy'])

In [0]:
batch_size=32
nb_epochs=10
data_generator_train = PascalVOCDataGenerator('trainval', data_dir)
steps_per_epoch_train = int(len(data_generator_train.id_to_label) / batch_size) + 1
model.fit_generator(data_generator_train.flow(batch_size=batch_size),
                    steps_per_epoch=steps_per_epoch_train,
                    epochs=nb_epochs,
                    verbose=1)

In [0]:
import numpy as np
from sklearn.metrics import average_precision_score
# import PascalVOCDataGenerator

default_batch_size = 200
default_data_dir = './content/data/VOCdevkit/VOC2007'

def evaluate(model, subset, batch_size=default_batch_size, data_dir=default_data_dir, verbose=1):
    """evaluate
    Compute the mean Average Precision metrics on a subset with a given model

    :param model: the model to evaluate
    :param subset: the data subset
    :param batch_size: the batch which will be use in the data generator
    :param data_dir: the directory where the data is stored
    :param verbose: display a progress bar or not, default is no (0)
    """
    #disable_tqdm = (verbose == 0)

    # Create the generator on the given subset
    data_generator = PascalVOCDataGenerator(subset, data_dir)
    steps_per_epoch = int(len(data_generator.id_to_label) / batch_size) + 1

    # Get the generator
    generator = data_generator.flow(batch_size=batch_size)

    y_all = []
    y_pred_all = []
    for i in range(steps_per_epoch):
        # Get the next batch
        X, y = next(generator)
        y_pred = model.predict(X)
        # We concatenate all the y and the prediction
        for y_sample, y_pred_sample in zip(y, y_pred):
            y_all.append(y_sample)
            y_pred_all.append(y_pred_sample)
    y_all = np.array(y_all)
    y_pred_all = np.array(y_pred_all)

    # Now we can compute the AP for each class
    AP = np.zeros(data_generator.nb_classes)
    for cl in range(data_generator.nb_classes):
        AP[cl] = average_precision_score(y_all[:, cl], y_pred_all[:, cl])
    MAP = AP.mean()*100
    
    return AP, MAP

In [0]:
subset = 'train'
AP, MAP = evaluate(model, subset)
print ("MAP  =", MAP)

In [0]:
subset = 'test'
AP, MAP = evaluate(model, subset)
print ("MAP  =", MAP)